In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Importar Sales Calendar
calendar = pd.read_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD input/daily_calendar_with_events.csv")

#Importar Item Sales
sales = pd.read_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD input/item_sales.csv")

#Importar Item Prices
prices = pd.read_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD input/item_prices.csv")

In [ ]:
sales

In [ ]:
#Comprobación que tienen las mismas fechas
calendar_transpose = calendar.set_index("d").T
#Lista de "d" de "calendar"
lista = calendar_transpose.columns.tolist()
#Lista de "d" de "sales"
lista_1 = sales.columns.values.tolist()
lista_1 = lista_1[7:]
assert (lista == lista_1)

In [ ]:
#Creación del diccionario con valores de "d" y de "date"
calendar_transpose_dates = calendar.set_index("date").T
lista_dates = calendar_transpose_dates.columns.tolist()
dict_dates = dict(zip(lista_1, lista_dates))

In [ ]:
#Cambio de nombre de las columnas en la base de datos de "sales"
sales.rename(columns=dict_dates, inplace=True)
sales.head()

In [ ]:
sales_nan = sales.replace(0, np.nan)
sales_nan = sales.dropna(how='all', axis=0)

In [ ]:
sales_nan

In [ ]:
#Creamos la lista de variables idenfiticadoras y variables de medidas.
lista = sales.columns.values.tolist()
lista_1 = lista[0:7]
del lista[0:7]
lista_1

In [ ]:
sales = sales.melt(id_vars=lista_1, var_name="date", value_name= "units")

In [ ]:
sales = sales.replace(0, np.nan)
sales.dropna(inplace= True)

In [ ]:
sales

In [ ]:
calendar["date1"] = pd.to_datetime(calendar["date"], format = "%Y-%m-%d")
calendar["week"] = calendar["date1"].dt.isocalendar().week.astype(str)
calendar["year"] = calendar["date1"].dt.isocalendar().year.astype(str)
calendar["year_week"] = calendar["year"] + "-" + calendar["week"]

In [ ]:
sales_calendar = pd.merge(
    left = sales,
    right = calendar,
    how = 'left',
    on = ["date"])

In [ ]:
sales_calendar

In [ ]:
sales_calendar["weekend_sales"] = np.where( (sales_calendar.weekday == "Saturday") | (sales_calendar.weekday == "Sunday"), sales_calendar.units, 0)

In [ ]:
sales_calendar["event_dummy"] = sales_calendar.event.isnull().value_counts()

In [ ]:
sales_calendar["event_sales"] = np.where( (sales_calendar.event_dummy == True), sales_calendar.units, 0)

In [ ]:
sales_calendar

In [ ]:
sales_calendar_weekly = sales_calendar.groupby(["id", "year", "week"]).sum()

In [ ]:
sales_calendar_weekly.reset_index(inplace =True)

In [ ]:
sales_calendar_weekly

In [ ]:
sales_calendar_weekly.to_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/database_weekends_events_4.csv")

In [ ]:
calendar["event"].value_counts()

In [ ]:
prueba = prices.groupby(["item"]).first()

In [ ]:
prueba = prices.groupby(["yearweek", "item", "store_code"]).first()

In [ ]:
prueba.reset_index(inplace = True)

In [ ]:
prueba_min = prueba.groupby(["yearweek", "item"]).min()

In [ ]:
prueba_max = prueba.groupby(["yearweek", "item"]).max()

In [ ]:
prueba_merge = pd.merge(
    left = prueba_min,
    right = prueba_max,
    how = 'left',
    on = ["yearweek", "item"])

In [ ]:
prueba_merge